# Natural Language Processing in Keras

Welcome! In this tutorial, I am going to cover a few approaches to NLP in the Keras framework. We have a **lot** of ground to cover, and I will try to cater to as broad a range of skill levels as I can manage. If you have any questions, do not hesitate to ask! I'm kind of making this up as I go along, so I am not sure how balanced the difficulty of each section is. We should have plenty of time to cover with ample questions. 

# What is NLP?

Natural language processing, or NLP, is a branch of computer science concerned with getting machines to understand and produce human language. Human language can be very tricky. Take the following [garden path sentences](https://en.wikipedia.org/wiki/Garden_path_sentence):

<br><center>**Time flies like an arrow. Fruit flies like a banana**</center>

In the first sentence, *flies* is a verb. In the second, the exact same word is a noun! That is why we humans find it humorous - it throws our brain in a direction it did not expect. To get this into machine-speak, we would have to do something like:

<br><center>**Time is similar to an arrow in that both move swiftly. Drosophila melanogaster enjoy consuming Musa acuminata**</center>

Kinda takes the fun out of it, huh? But that is exactly what we have to do when it comes to processing natural language with computers. We can understand language, in spoken or written form, often with ambiguous context, thanks to the exaflop processor sitting in our skull. Computers have one one-millionth of that to work with, so we are going to have to be clever. 

# The goal

Our primary task is fairly straightforward: given a written review of a movie, can we rate it as positive or negative? I am also working on a secondary challenge, which may or may not make it into the final tutorial. That challenge is, given a model trained on reviews from one source (IMDB), can we use it to accurately rate reviews from another source(Rotten Tomatoes)? The former is a toy example, which the latter has been a bit of a challenge for me! I'm experimenting as I go along, so even as I write this, I'm not sure how it'll turn out!

## Hurdles

There are two main hurdles in our way, and I will be covering some mainstream approaches to tackling them. The first is vocabulary: How do we take the 170,000+ words in the English language, and render that into a number which makes sense to a computer, and does not take a huge amount of information? This process is known as **embedding**. The second is context: As demonstrated in our above phrase, a word by itself is not guaranteed to have a single meaning. We have to look to the *context*, that is, the words and sentences around it. We will tackle this with **Convolutional networks** and **Recurrent networks**. 

Alright, let's look at some code! First, let's load in the libraries we will need.

In [1]:
# LSTM and CNN for sequence classification in the IMDB dataset
import numpy as np
import keras
from keras.datasets import imdb, reuters
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, Dropout, RepeatVector

from keras.layers.noise import GaussianNoise
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Convolution1D, MaxPooling1D

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras_tqdm import TQDMNotebookCallback


Using TensorFlow backend.


#### Keras recently rolled out V2.0 with some breaking changes. I'll try to keep this tutorial as v1/v2 compatible as possible in case people are still running v1. 
If you see errors like `TypeError: Received unknown keyword arguments: {'epochs': 3}`, or size/shape mismatches, you probably have a version mismatch

In [2]:
print(keras.__version__)

2.0.2


In [3]:
# Fix random seed for reproducibility
np.random.seed(7)

# Preprocess the dataset into suitable shape to feed to NN

We will load in the data into memory, and then process it from the current form into a form that our model can accept. 

In [4]:

nb_top_words           = 5000  # Load the dataset but only keep the top n words, zero the rest. These will be our 'symbols' in our abridged corpus
sequence_len           = 50   # This is the length in the "time axis". Originally was 500, but it turns out 50 works *almost* as well, and way faster!
embed_vector_len       = 32    # Size of the feature vector each word will map to
nb_lstm                = 100   # Number of LSTM nodes
batch_size             = 64    # Number of samples to feed into the model for each forward/backward pass

In [27]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(nb_words=nb_top_words, skip_top=0, index_from=3)#nb_words=nb_top_words)

/home/mike/ve/keras/lib/python3.5/site-packages/keras/datasets/imdb.py:44: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


In [37]:
twitter_x = np.load('data/twitter_phrases.npy') # pre-converted indexes from Twitter set using IMDB word list
twitter_y = np.load('data/twitter_labels.npy')
rotten_x = np.load('data/rotten_phrases.npy') # pre-converted indexes from Twitter set using IMDB word list
rotten_y = np.load('data/rotten_labels.npy')

## Let's peer into the data. I always like to start by getting a feel for the data

This data is in the form of a sequence of integers.  Each `int` maps to a word in the corpus dictionary. It is a giant lookup table, with the index roughly corresponding to the frequency rank of the word. This is necessary because computers do not directly process words, only numbers. 

In [46]:
word_index = imdb.get_word_index()
index_word = {value: key for (key, value) in word_index.items()} # flip key:value pairs to get the integer as the key
index_word.update({0: "~"})
list(index_word.items())[:10]

[(0, '~'),
 (1, 'the'),
 (2, 'and'),
 (3, 'a'),
 (4, 'of'),
 (5, 'to'),
 (6, 'is'),
 (7, 'br'),
 (8, 'in'),
 (9, 'it')]

Here is an example sample. Let's look at the raw data, and then the text it corresponds to. 

In [51]:
sample_num = 3
x_train[sample_num][:100]

array([263,   2, 182,   5,  17,  75,   2, 922,  36, 279, 131,   2,  17,
         2,  42,  17,  35, 921,   2, 192,   5,   2,   2,  19,   2, 217,
         2,   2, 537,   2,   2,   5, 736,  10,  10,  61, 403,   9,   2,
        40,  61,   2,   5,  27,   2, 159,  90, 263,   2,   2, 309,   8,
       178,   5,  82,   2,   4,  65,  15,   2, 145, 143,   2,  12,   2,
       537, 746, 537, 537,  15,   2,   4,   2, 594,   7,   2,  94,   2,
         2,   2,  11,   2,   4, 538,   7,   2, 246,   2,   9,   2,  11,
       635,  14,   9,  51, 408,  12,  94, 318,   2], dtype=int32)

In [55]:
' '.join([index_word[idx] for idx in x_train[sample_num]])

"comes and young to movie bad and dream from reason these and movie and it's movie so fi and enough to and and film and almost and and obviously and and to appears i i only human it and just only and to be and new made comes and and high in want to other and of their for and those i'm and that and obviously message obviously obviously for and of and brother br and make and and and this and of blood br and worst and it and this across as it when lines that make excellent and that there is and fantasy to and and film good br of and and and have into your whatever i i and and and be and this and and new be home all and film and lot br made and in at this of and how and in and some this and not all it and are of and and re is and and and i i worst more it is and and message made all and in does of nor of nor side be and and obviously know end and here to all tries in does of nor side of home br be indeed i i all it and in could is performance and and in of and br by br and its and and well

# wat.
Ok, I totally did not expect that. It seems super nonsensical. I dug into it and I think the situation is that the Keras IMDB data has already been processed with bag-of-words. I left this in here because it's important to remember that **Data science is a science**. Unexpected things happen all the time. 


#### Next, we need to pad the tensors out to the proper dimension in the time axis. 
Even though LSTM can handle variable length data, the backend still prefers rectangular tensors. We will pad/crop our variable-length movie reviews so that they are all exactly the same length. Once we have regularly-sized tensors, we are ready to build the model. 

In [38]:
x_train = sequence.pad_sequences(x_train, maxlen=sequence_len)
x_test = sequence.pad_sequences(x_test, maxlen=sequence_len)

In [39]:
twitter_x = sequence.pad_sequences(twitter_x, maxlen=sequence_len)
rotten_x = sequence.pad_sequences(rotten_x, maxlen=sequence_len)
print(x_train.shape, x_test.shape, twitter_x.shape, rotten_x.shape)

(25000, 50) (25000, 50) (1291, 50) (3928, 50)


# Without further ado, let's build an RNN in 5 lines of code. I'll walk you through each layer in detail. 

First, we initialize the model.

> ```python
model = Sequential()```

The [Keras Sequential Model](https://keras.io/getting-started/sequential-model-guide/) is based around building up the model layer-by-layer, like a cake. This is the easiest to graps for beginners, and works well, since many, if not most, neural networks can be represented this way. 
Calling model.add(layer) sticks the layer onto the topmost, and that becomes the new top. 



### Embedding

We start off with an efficient embedding layer which maps our vocabulary to a lower-dimensional space.

> ```python
model.add(Embedding(nb_top_words,                 
                    embed_vector_len, 
                    input_length=sequence_len
                   ))
```

#### One-hot encoding

Currently, we have every word mapped to some integer, which is great because the computer can parse it, but another problem arises: These numbers are sort of arbitrary. 55 (time) and 56 (she), have absolutely no conection, even though they are numerically close. Since neural networks are essentially giant (non)linear equations, this is not ideal. Larger numbers would get more 'weight' than smaller ones. What we need to do is map it in to some sort of categorical representation, where each word is on 'equal footing'. 

We achieve this by using a [*one hot* representation](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science). This is where you have an Nx1 vector, where the dimension associated with the integer has value 1 ('hot') and all other are 0. N is the size of your vocabulary. Each word would look like this:

[0,0,0,0,0,0,0,...,0,**1**,0,0,...,0,0,0]
<br>        ^ this is the $kth$ dim, where $k$ corresponds to the index in the table


Here is a toy example:

```
here    = [0,0,0,0,1,0]
is      = [0,1,0,0,0,0]
a       = [1,0,0,0,0,0]
toy     = [0,0,0,1,0,0]
example = [0,0,0,0,0,1]
```

Normally, you would have way more dimensions (typically thousands). Fortunately, Keras takes care of this automatically. We give it a sequence of M integers, and it'll automatically convert it to a NxM matrix.

Currently, the "shape" of each data vector is Nx1, where N = 5000. That's really problematic, due to the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) and also computational constraints. We want to transform our **uncondensed** and **sparse** data into a **compact** and **dense** representation, in our case a 32-vector. This process is known as embedding. **Word2Vec** is a popular word embedding system. Keras Embedding layer works very similarly, so if you want to know more, I suggest checking out how Word2Vec works. 

After embedding, we will have a much more manageable 32x50 matrix. 

### LSTM

> ```python
model.add(LSTM(nb_lstm))```


The LSTM (long short-term memory) cell is a neuron with memory. It accomplishes this by have a memory state, which can be written to and read from. It's like a tiny ~~casette tape~~  [~~floppy disk~~](http://i.imgur.com/Osxo1UF.jpg)  USB flash drive. In short, the inputs from the prior layer (mathematically) control gates. These gates determine whether to erase, write, and/or read from the memory cell. LSTMs have been covered really well in depth  in a lot of places. In particular I recommend the articles by [colah](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) and [Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). 

![LSTM](http://tc.sinaimg.cn/maxwidth.800/tc.service.weibo.com/cdn_images_1_medium_com/58ad765e09eacb5116c9dfc5897c7296.png)

### Dense

> ```python
model.add(Dense(1, activation='sigmoid'))```


Finally, we have a densely-connected layer. This is your "typical" neural network layer - each node from the prior layer connects to each node of the following. In this case, we are crunching down to a single node since we want a single answer - "Positive" or "Negative". We'll use a sigmoid activation to squash the output to the range 0-1. 

That's it for our network! Pretty simple, right? 

> ["Keras is so good that it is effectively cheating in machine learning"](https://news.ycombinator.com/item?id=13872670)

### Compiling 

> ```python
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])```

All that's left is compiling the model. This tells the model which loss function, optimizer, and metrics to use. 

- **Optimizer**: This is the algorithm which describes how we approach gradient descent. Adam is pretty modern and works quite well for a lot of problems, so it is typically the first go-to when picking hyperparameters
- **Metrics**: This does not actually affect the direct training of the model. Rather, it gives us humans a way to track the performance of the model over time. This can also be used for automatically early-stopping to avoid overfitting. 
- **Loss function**: This determines how the "penalty" for incorrect predictions is calculated. 

### Brief tangent: Cross-entropy
I'm working on a [simple summary of Cross-Entropy](https://github.com/xkortex/TechValleyMachineLearning/blob/master/CrossEntropy.ipynb). If you would like to know more, then check out that post, otherwise it's a bit of a tangent for this particular project. 

For now, all we need to know is cross-entropy is a very common loss function, and many Keras models use binary (yes/no problems) or categorical (multiple labels) cross-entropy. You are probably familiar with Mean Squared Error, which is a commonly used loss function if you are performing continuous regression. Cross-entropy is used to predict labels (logistic regression). The legendary [Andrew Ng Coursera Course](https://www.coursera.org/learn/machine-learning) covers this in more detail. 

# Let's write the code for the model


In [61]:
model = Sequential()
model.add(Embedding(nb_top_words,                 
                    embed_vector_len, 
                    input_length=sequence_len
                   ))
model.add(LSTM(nb_lstm))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# Have a look at our model
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 32)            160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301.0
Trainable params: 213,301
Non-trainable params: 0.0
_________________________________________________________________
None


# Fit the model to the data
This is where the magic happens! The Tensor Gnomes will do their ritual dance, and the weights will manifest new values.

...What do you mean, that's not how it works?

In [16]:
# This will take about 20-60 seconds per epoch on a GPU. CPU will take anywhere from 30s (i7 w/ 8 threads) to several minutes per epoch. 
# Larger networks will see a bigger advantage to GPU, though CPU did not do as poorly as I expected!
# We'll be making improvements to the model, so you don't have to run this just yet.
RUN_MODEL1 = True# I'm just using a switch here so I can run Kernel -> Restart and Run All
if RUN_MODEL1: 
    model.fit(x_train, y_train, 
              validation_data=(x_test, y_test), 
              epochs=3, 
              batch_size=batch_size, 
              verbose=0, # Some versions of Jupyter bork on Keras' progress bar. We replace it with Keras-TQDM instead
              callbacks=[TQDMNotebookCallback()])

This network will get us to about 87% accuracy. However, we had to stop pretty early because of the risk of overfitting. A model that overfits easy is often a strong sign that the model will generalize poorly to new, unseen data, or data from different distributions. Let's see how it performs on a similar dataset, the Twitter dataset. This will be challenging, as it is much shorter, which gives the RNN less time to 'get up to speed'.  

In [40]:
if RUN_MODEL1:
    loss, metric = model.evaluate(twitter_x, twitter_y)
    print('Label ratio: {:.2f}%'.format(np.mean(twitter_y*100)))
    print('Loss: {}\nMetric: {:.2f}%'.format(loss, metric*100))

1184/1291 [==========================>...] - ETA: 0sLabel ratio: 56.70%
Loss: 1.5329099910190915
Metric: 45.31%


#### Ouch. No better than chance.
Let's see if we can do a bit better with the generalization. Since the dataset we want to extrapolate to (Twitter, Rotten Tomatoes, etc) is different in several ways, we want our IMDB-trained network to be robust to noise, idiosyncracies, and quirks unique to IMDB that do not generalize to other formats.  

### Adding dropout and noise

**Dropout** is a technique for reducing overfitting in neural networks by preventing over-adaptation to the training data. The general idea is, every cycle you randomly drop a certain percentage of nodes or connections. This forces the network to compensate by distributing over multiple nodes and prevents any given node from getting too "specialized". 

Adding noise is another way of reducing overfitting and improving generalization. By adding noise to the vectors, this forces the network to learn to compensate, just as humans process stimuli in a noisy environment.

In [ ]:
dropout_rate = 0.2 # Rate of input units to drop
sigma=0.5         # Amount of noise to add (in terms of standard deviation)

model=Sequential()
model.add(Embedding(nb_top_words,                 
                    embed_vector_len, 
                    input_length=sequence_len
                   ))
model.add(GaussianNoise(sigma=sigma))
model.add(Dropout(dropout_rate))
model.add(LSTM(nb_lstm))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# This will take about 4-6 minutes per epoch on a GPU. 
RUN_MODEL2 = False
if RUN_MODEL2:
    model.fit(x_train, y_train, 
              validation_data=(x_test, y_test), 
              nb_epoch=3, 
              batch_size=batch_size, 
              verbose=0, # Some versions of Jupyter bork on Keras' progress bar. We replace it with Keras-TQDM instead
              callbacks=[TQDMNotebookCallback()])

In [ ]:
if RUN_MODEL2:
    loss, metric = model.evaluate(twitter_x, twitter_y)
    print('Label ratio: {:.2f}%'.format(np.mean(twitter_y*100)))
    print('Loss: {}\nMetric: {:.2f}%'.format(loss, metric*100))

# LSTM + CNN = REAL ULTIMATE POWER!

#### LSTMs are awesome. Convolutional neural networks are awesome. I was blown away when I learned you can simply and easily combine both into the same model!

Convolutional layers look at local structures in the data. In image classifiers, this is visual features. In NLP, this looks at groups of words, or N-grams (sequence of N words). For instance, many English sentences contain N-grams of the form [**subject verb object**]. For example (dropping articles for simplicity):

- John rode bike
- Suzie hit ball
- Bobby made memes

Another common sequence is [**subject copula predicate**] (copula verbs: is, are, was, will be: 

- Roses are red
- Movie was bad
- Keras is awesome

In [24]:
nb_filter = 256     # This is the number of convolutional filters to use
filter_length = 5  # This is the size of the filter kernel. Since this is 1D, the kernel is Nx1
pool_length = 2    # Size of our max pooling structures
dropout_rate = 0.2 # Rate of input units to drop
sigma=0.5         # Amount of noise to add (in terms of standard deviation)

model = Sequential()
model.add(Embedding(nb_top_words, embed_vector_len, input_length=sequence_len))
model.add(GaussianNoise(sigma=sigma))
model.add(Convolution1D(nb_filter=nb_filter, filter_length=filter_length, border_mode='same', activation='relu'))
model.add(MaxPooling1D(pool_length=pool_length))
model.add(Dropout(dropout_rate))
model.add(LSTM(100))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/home/mike/ve/keras/lib/python3.5/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `GaussianNoise` call to the Keras 2 API: `GaussianNoise(stddev=0.5)`
/home/mike/ve/keras/lib/python3.5/site-packages/ipykernel/__main__.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(kernel_size=5, padding="same", filters=256, activation="relu")`
/home/mike/ve/keras/lib/python3.5/site-packages/ipykernel/__main__.py:11: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`


In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 50, 32)            160000    
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 50, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 256)           41216     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 25, 256)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 256)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               142800    
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
__________

## some stuff about sttructure

In [36]:
# This will take about 0.5-2 minutes per epoch on a GPU. 
RUN_MODEL3 = True
if RUN_MODEL3:
    model.fit(x_train, y_train, 
              validation_data=(x_test, y_test), 
              nb_epoch=3, 
              batch_size=batch_size, 
              verbose=0, # Some versions of Jupyter bork on Keras' progress bar. We replace it with Keras-TQDM instead
              callbacks=[TQDMNotebookCallback()])
    
loss, metric = model.evaluate(x_test, y_test, verbose=0)
print('Final validation accuracy: {:.2f}%'.format(metric*100))

/home/mike/ve/keras/lib/python3.5/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '



Final validation accuracy: 81.49%


In [41]:
if RUN_MODEL3:
    loss, metric = model.evaluate(twitter_x, twitter_y)
    print('Label ratio: {:.2f}%'.format(np.mean(twitter_y*100)))
    print('Loss: {}\nMetric: {:.2f}%'.format(loss, metric*100))
    loss, metric = model.evaluate(rotten_x, rotten_y)
    print('Label ratio: {:.2f}%'.format(np.mean(rotten_y*100)))
    print('Loss: {}\nMetric: {:.2f}%'.format(loss, metric*100))

1152/1291 [=========================>....] - ETA: 0sLabel ratio: 56.70%
Loss: 1.5329099910190915
Metric: 45.31%
3808/3928 [============================>.] - ETA: 0s

In [ ]:
nb_top_words           = 20000  # Load the dataset but only keep the top n words, zero the rest. These will be our 'symbols' in our abridged corpus
(x_train, y_train), (x_test, y_test) = imdb.load_data(nb_words=nb_top_words, skip_top=0, index_from=3)#nb_words=nb_top_words)
x_train = sequence.pad_sequences(x_train, maxlen=sequence_len)
x_test = sequence.pad_sequences(x_test, maxlen=sequence_len)

In [ ]:
model = Sequential()
model.add(Embedding(nb_top_words, embed_vector_len, input_length=sequence_len))
model.add(GaussianNoise(sigma=sigma))
model.add(Dropout(dropout_rate))
model.add(LSTM(nb_lstm))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
RUN_MODEL4 = True
if RUN_MODEL4:
    model.fit(rotten_x, rotten_y, 
              validation_data=(x_test, y_test), 
              nb_epoch=10, 
              batch_size=batch_size, 
              verbose=0, # Some versions of Jupyter bork on Keras' progress bar. We replace it with Keras-TQDM instead
              callbacks=[TQDMNotebookCallback()])
    
loss, metric = model.evaluate(x_test, y_test, verbose=0)
print('Final validation accuracy: {:.2f}%'.format(metric*100))

In [ ]:
model.fit(x_train, y_train, 
              validation_data=(x_test, y_test), 
              nb_epoch=10, 
              batch_size=batch_size, 
              verbose=0, # Some versions of Jupyter bork on Keras' progress bar. We replace it with Keras-TQDM instead
              callbacks=[TQDMNotebookCallback()])

In [ ]:
if RUN_MODEL4:
    loss, metric = model.evaluate(x_test, y_test, verbose=0)
    print('IMDB:\nLabel ratio: {:.2f}%'.format(np.mean(twitter_y*100)))
    print('Loss: {}\nMetric: {:.2f}%'.format(loss, metric*100))
    loss, metric = model.evaluate(twitter_x, twitter_y)
    print('Twitter:\nLabel ratio: {:.2f}%'.format(np.mean(twitter_y*100)))
    print('Loss: {}\nMetric: {:.2f}%'.format(loss, metric*100))
    loss, metric = model.evaluate(rotten_x, rotten_y)
    print('Rotten Tomatoes:\nLabel ratio: {:.2f}%'.format(np.mean(rotten_y*100)))
    print('Loss: {}\nMetric: {:.2f}%'.format(loss, metric*100))

In [ ]:
input_dim = nb_top_words
latent_dim = 24
inputs = Input(shape=(sequence_len, input_dim))
encoded = LSTM(latent_dim)(inputs)
decoded = RepeatVector(sequence_len)(encoded)
decoded = LSTM(input_dim, return_sequences=True)(decoded)
sequence_autoencoder = Model(inputs, decoded)
encoder = Model(inputs, encoded)

In [ ]:
print(sequence_autoencoder.summary())

# References

## Datasets

http://www.cs.cornell.edu/people/pabo/movie-review-data/
http://snap.stanford.edu/data/web-Amazon.html

## Papers
- [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
- [Semi-supervised Sequence Learning](https://arxiv.org/abs/1511.01432)
- [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)

## Code

- [Keras Examples: IMDB CNN](https://github.com/fchollet/keras/blob/master/examples/imdb_cnn.py)
- [Machine Learning Mastery: Sequence Classification with LSTM Recurrent Neural Networks in Python with Keras](http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/)
- [Microway: Building a Movie Review Sentiment Classifier using Keras and Theano Deep Learning Frameworks](https://www.microway.com/hpc-tech-tips/keras-theano-deep-learning-frameworks/)
- [Using  Pre-trained Word Embeddings in Keras](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html)

## Tangentally related

- [McCormick: Word2vec Tutorial](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)

# Bonus feature: Pure CNN Model

I was not sure how much ground I would be able to cover, so this section is here in case people are having a great time, and are really thirsty for knowledge! This is a pure-CNN based model, and works by essentially dumping all the N-grams together (global pooling). It does not care about the respective order of N-grams. And yet, it works quite well! 

In [42]:
'''This example demonstrates the use of Convolution1D for text classification.
Gets to 0.89 test accuracy after 2 epochs.
90s/epoch on Intel i5 2.4Ghz CPU.
10s/epoch on Tesla K40 GPU.
'''

from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 1000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...


/home/mike/ve/keras/lib/python3.5/site-packages/keras/datasets/imdb.py:44: UserWarning: The `nb_words` argument in `load_data` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `load_data` '


25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [43]:
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu'))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [44]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 400, 50)           50000     
_________________________________________________________________
dropout_5 (Dropout)          (None, 400, 50)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 398, 250)          37750     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 250)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_6 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_3 (Activation)    (None, 250)               0         
__________

In [20]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test), verbose=0, callbacks=[TQDMNotebookCallback()])